In [1]:
import pandas as pd
import numpy as np
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

data_path = '../data/'
df_train = pd.read_csv(data_path + 'train_offline.csv')
df_test = pd.read_csv(data_path + 'test_offline.csv')
#train_Y = pd.read_csv(data_path + 'trainLabels.csv',header=None)
df_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [2]:
def label(row):
    if(np.isnan(row['Date_received'])):
        return -1
    else:
        if(~np.isnan(row['Date'])):
            Day_Diff = pd.to_datetime(row['Date'],format='%Y%m%d')-pd.to_datetime(row['Date_received'],format='%Y%m%d')
            if(Day_Diff<=pd.Timedelta(15,'D')):
                return 1
            else:
                return 0
        else:
            return 0
'''
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
'''
df_train['Label'] = df_train.apply(label,axis = 1)

Training Set 中只有Discount_rate，Distance和Date_received三個columns可以做為估計變量<br>
Merchant_id需要檢查unique數，如果商家數不多，也可以做為指標。

In [9]:
def disount_cost(row):
    if(row['Discount_rate']=='nan'):
        return 'nan'
    else:
        if(len(row['Discount_rate'].split(':'))==1):
            return 'nan'
        else:
            return row['Discount_rate'].split(':')[0]


df_train['Discount_Cost'] = df_train.astype(str).apply(disount_cost,axis = 1)
print(df_train['Discount_Cost'].value_counts())

def disount_discount(row):
    if(row['Discount_rate']=='nan'):
        return 'nan'
    else:
        if(len(row['Discount_rate'].split(':'))==1):
            return 'nan'
        else:
            return row['Discount_rate'].split(':')[1]


df_train['Discount_Discount'] = df_train.astype(str).apply(disount_discount,axis = 1)
print(df_train['Discount_Discount'].value_counts())

def discount_ratio(row):
    if(row['Discount_rate']=='nan'):
        return np.nan
    else:
        if(len(row['Discount_rate'].split(':'))==1):
            return float(row['Discount_rate'])
        else:
            return 1-float(row['Discount_rate'].split(':')[1])/float(row['Discount_rate'].split(':')[0])
df_train['Discount_Ratio'] = df_train.astype(str).apply(discount_ratio,axis = 1)
print(df_train['Discount_Ratio'].value_counts())

nan    441232
100    204677
30     180897
200    138079
50      65345
20      63434
300     28918
10      27889
150      9670
5         601
Name: Discount_Cost, dtype: int64
nan    441232
5      278240
10     199749
20     117648
30      86224
1       32412
50       5228
100         9
Name: Discount_Discount, dtype: int64
0.900000    354273
0.833333    172602
0.750000     52140
0.800000     33864
0.950000     33359
0.700000     30234
0.850000     27497
0.500000     21844
0.966667      6456
0.933333      4958
0.866667      4411
0.980000      2331
0.666667      1889
0.600000       706
0.990000       240
0.200000       110
0.975000        38
0.400000         9
0.333333         8
Name: Discount_Ratio, dtype: int64


In [28]:
df_train['Discount_Discount'] = df_train['Discount_rate'].astype(str).apply(lambda x: np.nan if x=='nan' else float(x.split(':')[1]))

IndexError: list index out of range

In [33]:
df_train['Discount_rate'].astype(str)[1].split(':')[1]

'1'

In [37]:
df_train['Discount_Cost'].value_counts()

100.00    204677
30.00     180897
200.00    138079
50.00      65345
20.00      63434
300.00     28918
10.00      27889
0.95       17392
150.00      9670
0.90        7226
0.80        1734
0.85         633
5.00         601
0.50         178
0.20         110
0.75          88
0.60          54
0.70          44
Name: Discount_Cost, dtype: int64

In [33]:
temp.apply(lambda x: 1 if np.isnan(x) else int(x))

0           1
1    20160217
2    20160319
3    20160429
4    20160129
Name: Date_received, dtype: int64

In [53]:
df_train['Label'].value_counts()

 0    704033
-1    413773
 1     42936
Name: Label, dtype: int64

In [55]:
df_train['Label'].value_counts()

 0    710665
-1    413773
 1     36304
Name: Label, dtype: int64

In [8]:
df_train['Label'].value_counts()

 0    710665
-1    413773
 1     36304
Name: Label, dtype: int64

In [7]:
df_train['Discount_Cost'].value_counts()

nan    441232
100    204677
30     180897
200    138079
50      65345
20      63434
300     28918
10      27889
150      9670
5         601
Name: Discount_Cost, dtype: int64